# Lab: Visualizing Learning With Tensorboard

### Overview
Introducing a suit of visualization tools

### Runtime
30 mins


# MNIST Dataset Introduction

Most examples are using MNIST dataset of handwritten digits. The dataset contains 60,000 examples for training and 10,000 examples for testing. The digits have been size-normalized and centered in a fixed-size image (28x28 pixels) with values from 0 to 1. For simplicity, each image has been flatten and converted to a 1-D numpy array of 784 features (28*28).

## Overview

![MNIST Digits](http://neuralnetworksanddeeplearning.com/images/mnist_100_digits.png)

## Usage
In our examples, we are using TensorFlow [input_data.py](https://github.com/tensorflow/tensorflow/blob/r0.7/tensorflow/examples/tutorials/mnist/input_data.py) script to load that dataset.
It is quite useful for managing our data, and handle:

- Dataset downloading

- Loading the entire dataset into numpy array: 




## Start

In [1]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False
if IN_COLAB == True:
    !pip install -U tensorboardcolab
    from tensorboardcolab import *
    tbc=TensorBoardColab()


#### In this section we are going to download the dataset from the website

In [2]:
from __future__ import print_function

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


#### Then we need to set and define all parameteres needed for the lab

In [3]:
learn_rate = 0.01                                                              
train_epochs = 25                                                           
batch_size = 100
display_epoch = 1
logs_path = '/tmp/tensorflow_logs/example/'

# mnist data image : 28*28=784
x = tf.placeholder(tf.float32, [None, 784], name='Input')

# We have 0-9 digits 
y = tf.placeholder(tf.float32, [None, 10], name='Label')

# Setting model weights
W = tf.Variable(tf.zeros([784, 10]), name='Weight')
b = tf.Variable(tf.zeros([10]), name='Bias')

Instructions for updating:
Colocations handled automatically by placer.


#### Now it's time to construct the model. Encapsulation will be applied for TensorBoard graph 

In [4]:
with tf.name_scope('Model'):
    # Model with softmax algorithm
    pred = tf.nn.softmax(tf.matmul(x, W) + b)
    
with tf.name_scope('Loss'):
    # Minimizing error using cross entropy
    cost = tf.reduce_mean(-tf.reduce_sum(y * tf.log(pred), reduction_indices=1))
    
with tf.name_scope('SGD'):
    # Using gradient descent
    optimizer = tf.train.GradientDescentOptimizer(learn_rate).minimize(cost)
    
with tf.name_scope('Accuracy'):
    # Accuracy
    acc = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

# Creating a summary to monitor cost tensor
tf.summary.scalar("loss", cost)

# Creating a summary to monitor accuracy tensor
tf.summary.scalar("accuracy", acc)

# Creating a summary to monitor weight tensor
tf.summary.histogram("Weight", W)

# Creating a summary to monitor bias tensor
tf.summary.histogram("Bias", b)

# Merging all summaries into a single op
merged_summary_op = tf.summary.merge_all()

Instructions for updating:
Use tf.cast instead.


### Training time

In [5]:
with tf.Session() as sess:
    sess.run(init)

    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

    # Training cycle
    for epoch in range(train_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples / batch_size)
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            
            # Run optimization op (backprop), cost op (to get loss value)
            # and summary nodes
            _, c, summary = sess.run([optimizer, cost, merged_summary_op],
                                     feed_dict={x: batch_xs, y: batch_ys})
            
            # Writing logs at each iteration
            
            if IN_COLAB:
                summary_writer = tbc.get_writer()# saving data on google colab
                summary_writer.add_summary(summary, epoch * total_batch + i)
            
            # Compute average loss
            avg_cost += c / total_batch
            
        # Displaying logs per epoch step
        if (epoch+1) % display_epoch == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")
    print("Accuracy:", acc.eval({x: mnist.test.images, y: mnist.test.labels}))

Epoch: 0001 cost= 1.183620099
Epoch: 0002 cost= 0.665360624
Epoch: 0003 cost= 0.552858688
Epoch: 0004 cost= 0.498707141
Epoch: 0005 cost= 0.465490984
Epoch: 0006 cost= 0.442524161
Epoch: 0007 cost= 0.425546776
Epoch: 0008 cost= 0.412213841
Epoch: 0009 cost= 0.401406325
Epoch: 0010 cost= 0.392411532
Epoch: 0011 cost= 0.384753361
Epoch: 0012 cost= 0.378176026
Epoch: 0013 cost= 0.372430801
Epoch: 0014 cost= 0.367312132
Epoch: 0015 cost= 0.362740347
Epoch: 0016 cost= 0.358556446
Epoch: 0017 cost= 0.354845505
Epoch: 0018 cost= 0.351478194
Epoch: 0019 cost= 0.348365503
Epoch: 0020 cost= 0.345428882
Epoch: 0021 cost= 0.342740956
Epoch: 0022 cost= 0.340229057
Epoch: 0023 cost= 0.337873283
Epoch: 0024 cost= 0.335752265
Epoch: 0025 cost= 0.333674024
Optimization Finished!
Accuracy: 0.9136


### Now we run the TensorBaord

#### Local:

Now run the following command on a bash

```bash
$ tensorboard --logdir=/tmp/tensorflow_logs
```
Then open the browser(Chrome) and insert this address

```bash
localhost:6006
```

#### Google colab


Click on the link you had got at the first step

![](../images/tb_colab.png)
